In [1]:
# import all necessary libraries including BeautifulSoup and requests for 
# reading in html text from Wikipedia webpage and parsing the text for the 
# table of Toronto area postcodes, boroughs and neighborhoods

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

!conda install -c conda-forge lxml --yes
import lxml

import pandas as pd
import numpy as np

import requests

from sklearn.cluster import KMeans


Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [33]:
# get html object via request
wikipage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse html object using BeautifulSoup and lxml
soup = BeautifulSoup(wikipage,'lxml')

# identify portion of the html text containing the table of Toronto postcodes
table = soup.find('table')

# loop through the list of all table entries (identified via tag 'tr')
# and extract and format each row of the table, making sure to split
# each table row by carriage return '\n' and removing any whitespace
tablebody = []
for xx in table.find_all('tr'):
    tablebody.append(xx.text.split('\n')[1:-1])

# create pandas DataFrame to store table    
df = pd.DataFrame(tablebody[1:],columns=tablebody[0])

# only keep those rows that have a Borough identified by name
df = df.loc[df['Borough'] != 'Not assigned',:]

# replace the neighbourhood name with the borough name for those 
# neighborhoods with unassigned names
df.loc[df['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df.loc[df['Neighbourhood'] == 'Not assigned','Borough']

# define function to concatenate a list of names into a list
def f2(x):
    return(list(x.unique()))

# group Dataframe by borough and apply function to Postcode and Neighborhood columns
df = df.groupby(['Postcode']).agg({'Borough':f2, 'Neighbourhood': f2}).reset_index()

print('Number of rows in dataframe (i.e. number of postcodes) ', df.shape[0])

Number of rows in dataframe (i.e. number of postcodes)  103
